In [17]:
import csv
import os
unique_entities_strict = set()
unique_entities_lenient = set()
entities = []

text = []
labels = []
labels_lenient = []
working_folder = "./custom_output/"

with open(os.path.join(working_folder, "ner-crf-training-data.tsv"), newline = '') as lines:  
#     line_reader = csv.reader(lines, delimiter='\t')
    line_reader = csv.reader(lines, delimiter='\t', quoting=csv.QUOTE_NONE)
    sentence = []
    entities = []
    pre_entity = "NEVER_EXISTS_ENTITY"
    for idx, line in enumerate(line_reader):
        ## . signal the begining of a new line
        ## skip empty line
        if line == []:
            continue
        if line[0] == '.' and line[1] == "O":
            text.append(" ".join(sentence + [line[0]]))
            labels.append(" ".join(entities + [line[1]]))
            sentence, entities = [] , []
            strict_entity = line[1]
        else:
            sentence.append(line[0])
            if line[1] != "O" and pre_entity == line[1]:
                entities.append("I-"+line[1])
                strict_entity = "I-"+line[1]
            elif line[1] != "O" and pre_entity != line[1]:
                entities.append("B-"+line[1])
                strict_entity = "B-"+line[1]
            else:
                entities.append(line[1])
                strict_entity = line[1]
        
        
        labels_lenient.append(line[1])
        
        unique_entities_strict.add(strict_entity)
        unique_entities_lenient.add(line[1])
        entities.append(line[1])
        pre_entity = line[1]


In [ ]:
import pandas as pd
new_df = pd.DataFrame({"text": text, "labels": labels})
new_df.head()
new_df.to_csv(os.path.join(working_folder, "ner.csv"))

In [ ]:
print(f"Unique entities count:\nStrict: {len(unique_entities_strict)}\nLenient: {len(unique_entities_lenient)}")


In [ ]:
print(unique_entities_strict)

In [ ]:
unique_entities_lenient

Visualize data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sentence_length = [len(str.split()) for str in text]
                   



sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.2)
plt.figure(figsize=(8,6))

# Plot the distribution of comment lengths.
sns.distplot(sentence_length, kde=False, rug=False,color='plum', bins=50)

plt.title('Sentence Lengths')
plt.xlabel('Sentence Length')
plt.ylabel('# of Sentences');

In [ ]:
from collections import Counter
import itertools
all_labels = list(itertools.chain(*[l.split() for l in labels]))
label_count = dict(Counter(all_labels))
tmp_df = pd.DataFrame({"entity": label_count.keys(), "count": label_count.values()})
print(tmp_df.head())

sns.set(font_scale=0.8)
plt.figure(figsize=(20,5))
ax = sns.countplot(all_labels)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");


In [ ]:
from collections import Counter
import itertools

label_count = dict(Counter(labels_lenient))
tmp_df = pd.DataFrame({"entity": label_count.keys(), "count": label_count.values()})
print(tmp_df)

sns.set(font_scale=0.8)
plt.figure(figsize=(20,5))
ax = sns.countplot(labels_lenient)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");